In [1]:
import pandas as pd
import numpy as np
path_sub = "./sub/"
path_build = "./build/"
from utils import *
users = pd.read_pickle(f"{path_build}/train_user.pkl")
users.sort_values(by=['user_id'], ascending=[True], inplace=True)
users = users.reset_index(drop=True)

from sklearn.model_selection import KFold

def fold5_mean(filename):
    gender_preds = []
    age_preds = []
    kfolder = KFold(n_splits=5, shuffle=True, random_state=2020)
    kfold = kfolder.split(users[0:3000000])
    i = 1
    gender_oof = np.zeros((3000000, 2))
    age_oof = np.zeros((3000000, 10))

    for train_index, vali_index in kfold:
        print(i,train_index, vali_index)
        age_prob = np.load(f"{path_sub}/{filename}/{i}/age_prob.npy")
        gender_prob = np.load(f"{path_sub}/{filename}/{i}/gender_prob.npy")
        val_age_prob = np.load(f"{path_sub}/{filename}/{i}/val_age_prob.npy")
        val_gender_prob = np.load(f"{path_sub}/{filename}/{i}/val_gender_prob.npy")
        
        gender_preds.append(gender_prob)
        age_preds.append(age_prob)
        gender_oof[vali_index] = val_gender_prob
        age_oof[vali_index] = val_age_prob
        
        
        i = i+1
    gender_prob = np.array(gender_preds).mean(axis=0)
    age_prob = np.array(age_preds).mean(axis=0)
    
    age_accuracy = accuracy_score(y_true=users['age'][0:3000000]-1, y_pred=age_oof.argmax(axis=1))
    gender_accuracy = accuracy_score(y_true=users['gender'][0:3000000]-1, y_pred=gender_oof.argmax(axis=1))
    print(f'age accuracy_score:{age_accuracy}')
    print(f'gender accuracy_score:{gender_accuracy}, sum:{age_accuracy+gender_accuracy}')
    print(gender_prob.sum(), age_prob.sum())
    print('----------------------------------------------')
    return gender_prob, age_prob, gender_oof,age_oof

    
# 两个五折
gender_prob_51, age_prob_51, gender_oof_51, age_oof_51 = fold5_mean('sub-5-5fold')
gender_prob_52, age_prob_52, gender_oof_52, age_oof_52 = fold5_mean('sub-5times-5fold')
#gender_prob_53, age_prob_53, gender_oof_53, age_oof_53 = fold5_mean('sub-10-5fold')
gender_prob_54, age_prob_54, gender_oof_54, age_oof_54 = fold5_mean('sub-5unit*1-5fold')


# # LR
# gender_prob_lr = np.load(f"{path_sub}/lr/gender_prob.npy")
# age_prob_lr = np.load(f"{path_sub}/lr/age_prob.npy")
# gender_val_prob_lr = np.load(f"{path_sub}/lr/val_gender_prob.npy")
# age_val_prob_lr = np.load(f"{path_sub}/lr/val_age_prob.npy")

# 单模

age_prob1 = np.load(f"{path_sub}/sub-all_single_model/age_prob.npy")
gender_prob1 = np.load(f"{path_sub}/sub-all_single_model/gender_prob.npy")
val_age_prob1 = np.load(f"{path_sub}/sub-all_single_model/val_age_prob.npy")
val_gender_prob1 = np.load(f"{path_sub}/sub-all_single_model/val_gender_prob.npy")


data = pd.DataFrame()
data['sub-5-5fold_age'] = np.argmax(age_prob_51,axis=1) + 1
data['sub-5-5fold_gender'] = np.argmax(gender_prob_51,axis=1) + 1


data['sub-5times-5fold_age'] = np.argmax(age_prob_52,axis=1) + 1
data['sub-5times-5fold_gender'] = np.argmax(gender_prob_52,axis=1) + 1


data['sub-5unit*1-5fold_age'] = np.argmax(age_prob_54,axis=1) + 1
data['sub-5unit*1-5fold_gender'] = np.argmax(gender_prob_54,axis=1) + 1


data['sub-all_single_model_age'] = np.argmax(age_prob1,axis=1) + 1
data['sub-all_single_model_gender'] = np.argmax(gender_prob1,axis=1) + 1

data.to_csv('all_model.csv',index=False, encoding='utf-8')



1 [      1       6       8 ... 2999997 2999998 2999999] [      0       2       3 ... 2999990 2999993 2999994]
2 [      0       1       2 ... 2999997 2999998 2999999] [      9      24      30 ... 2999984 2999987 2999989]
3 [      0       1       2 ... 2999994 2999995 2999998] [      6       8      13 ... 2999996 2999997 2999999]
4 [      0       1       2 ... 2999997 2999998 2999999] [     10      11      16 ... 2999991 2999992 2999995]
5 [      0       2       3 ... 2999996 2999997 2999999] [      1      12      20 ... 2999976 2999983 2999998]
age accuracy_score:0.49995266666666666
gender accuracy_score:0.9480696666666667, sum:1.4480223333333333
1000000.0 999999.25
----------------------------------------------
1 [      1       6       8 ... 2999997 2999998 2999999] [      0       2       3 ... 2999990 2999993 2999994]
2 [      0       1       2 ... 2999997 2999998 2999999] [      9      24      30 ... 2999984 2999987 2999989]
3 [      0       1       2 ... 2999994 2999995 2999998] [  